In [2]:
pip install pandas

  Obtaining dependency information for pandas from https://files.pythonhosted.org/packages/16/c6/75231fd47afd6b3f89011e7077f1a3958441264aca7ae9ff596e3276a5d0/pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl.metadata
  Using cached pandas-2.2.2-cp311-cp311-macosx_11_0_arm64.whl.metadata (19 kB)
  Obtaining dependency information for pytz>=2020.1 from https://files.pythonhosted.org/packages/9c/3d/a121f284241f08268b21359bd425f7d4825cffc5ac5cd0e1b3d82ffd2b10/pytz-2024.1-py2.py3-none-any.whl.metadata
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Obtaining dependency information for tzdata>=2022.7 from https://files.pythonhosted.org/packages/65/58/f9c9e6be752e9fcb8b6a0ee9fb87e6e7a1f6bcab2cdc73f02bb7ba91ada0/tzdata-2024.1-py2.py3-none-any.whl.metadata
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 167.0 kB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 505.5/505.5 kB 72.0 kB/s

In [4]:
import pandas as pd
import numpy as np
import time

In [5]:
symbol = 'BTC/USDT'
timeframe = '1h'
ma_short_period = 20
ma_long_period = 50
balance_percentage = 0.1

In [7]:
data = pd.read_csv("/Users/karimmeguenni-tani/Bot project trading/Cryptobot/DataBase/1h/BTC-USDT-USDT.csv")
data.index = pd.to_datetime(data['date'], unit='ms')
data = data.drop('date', axis=1)


In [8]:
data

,open,high,low,close,volume
date,,,,,
2019-07-10 11:00:00,13073.5,13075.5,13044.5,13064.5,4.8952
2019-07-10 12:00:00,13064.5,13064.5,12900.5,13029.0,684.5329
2019-07-10 13:00:00,13029.0,13127.5,12996.5,13039.5,812.2584
2019-07-10 14:00:00,13039.5,13065.5,12549.5,12549.5,679.7040
2019-07-10 15:00:00,12549.5,12552.0,12121.0,12362.0,715.6550
...,...,...,...,...,...
2024-04-19 23:00:00,63964.2,64095.3,62890.7,63771.2,14875.2400
2024-04-20 00:00:00,63771.2,64216.4,63365.5,63401.3,8151.1100
2024-04-20 01:00:00,63401.3,63741.9,63031.2,63606.8,7683.9050


In [16]:
def calculate_ma(data):
    data['ma_short'] = data['close'].rolling(window=ma_short_period).mean()
    data['ma_long'] = data['close'].rolling(window=ma_long_period).mean()
    return data

In [10]:
def trading_strategy(data):
    if data['ma_short'].iloc[-1] > data['ma_long'].iloc[-1] and data['ma_short'].iloc[-2] <= data['ma_long'].iloc[-2]:
        return 'buy'
    elif data['ma_short'].iloc[-1] < data['ma_long'].iloc[-1] and data['ma_short'].iloc[-2] >= data['ma_long'].iloc[-2]:
        return 'sell'
    else:
        return 'hold'

In [18]:
trade_records={"wallet":0,"symbol":"BTC/USDT","Coin":"BTC","trade_status":"Buy","value_usdt":0,"value_coin":0,"date":time.time(),"timeframe":"1h"}
print(list(trade_records.keys()))
historical_trades_records = pd.DataFrame(columns=list(trade_records.keys()))
historical_trades_records

['wallet', 'symbol', 'Coin', 'trade_status', 'value_usdt', 'value_coin', 'date', 'timeframe']


,wallet,symbol,Coin,trade_status,value_usdt,value_coin,date,timeframe


In [29]:
def calculate_value_wallet(wallet,data):
    return wallet['usdt'] + wallet['coin'] * data['close'].iloc[-1]

In [33]:
condition=True

wallet={"usdt":1000,"coin":0}
parameters={"wallet":calculate_value_wallet(wallet,data),"taker_fee":0.1,"maker_fee":0.1,"balance_percentage_usdt":0.1,"balance_percentage_coin":1,"symbol":"BTC/USDT","num_trades":0,"trade_status":"None"}
parameters_records = pd.DataFrame(parameters, index=[0])
trade_records={"symbol":"BTC/USDT","Coin":"BTC","trade_status":"Buy","value_usdt":0,"value_coin":0,"date":time.time(),"timeframe":"1h"}
historical_trades_records = pd.DataFrame(columns=list(trade_records.keys()))

data = calculate_ma(data)
# Create a new dataframe with a single row using the parameters dictionary
while condition:
    # Obtenir l'heure actuelle
    current_time = time.strftime("%Y-%m-%d %H:%M:%S", time.localtime())

    # Vérifier si l'on doit acheter ou vendre
    action = trading_strategy(data)
    print("value action:",action)
    # Effectuer les calculs d'achat ou de vente en fonction des paramètres
    if action == 'buy' and parameters['wallet']['usdt'] > 0 and parameters['trade_status'] != 'Bought':
        # Calculer la valeur de l'achat
        value_usdt = (1 - parameters['taker_fee']) * parameters['balance_percentage_usdt'] * parameters['wallet']['usdt']
        value_coin = value_usdt / data['close'].iloc[-1]
        # Mettre à jour les paramètres
        wallet={"usdt":value_usdt,"coin":value_coin}
        parameters['wallet'] = calculate_value_wallet(wallet,data)
        parameters['num_trades'] += 1
        parameters['trade_status'] = 'Bought'
        # Stocker les résultats dans parameters_records
        parameters_records = parameters_records.append(parameters, ignore_index=True)
        # Stocker les résultats dans trade_records
        trade_records = {'symbol': parameters['symbol'], 'trade_status': 'Buy', 'value_usdt': value_usdt, 'value_coin': value_coin, 'date': current_time, 'timeframe': parameters['timeframe']}
        # Stocker les résultats dans historical_trades_records
        historical_trades_records = historical_trades_records.append(trade_records, ignore_index=True)
    elif action == 'sell' and parameters['wallet']['coin'] > 0:
        # Calculer la valeur de la vente
        value_coin =  (1 - parameters['maker_fee']) * (1-parameters['balance_percentage_coin']) * parameters['wallet']['coin']
        value_usdt =  value_coin * data['close'].iloc[-1]
        # Mettre à jour les paramètres
        parameters['wallet'] = calculate_value_wallet(wallet,data)
        parameters['num_trades'] += 1
        parameters['trade_status'] = 'Sold'
        # Stocker les résultats dans parameters_records
        parameters_records = parameters_records.append(parameters, ignore_index=True)
        # Stocker les résultats dans trade_records
        trade_records = {'symbol': parameters['symbol'], 'trade_status': 'Sell', 'value_usdt': value_usdt, 'value_coin': value_coin, 'date': current_time, 'timeframe': parameters['timeframe']}
        display(trade_records)
        # Stocker les résultats dans historical_trades_records
        historical_trades_records = historical_trades_records.append(trade_records, ignore_index=True)
    
    # Attendre une heure avant la prochaine itération
    condition=False


value action: hold


In [32]:
historical_trades_records

,symbol,Coin,trade_status,value_usdt,value_coin,date,timeframe


In [31]:
parameters_records

,wallet,taker_fee,maker_fee,balance_percentage_usdt,balance_percentage_coin,symbol,num_trades,trade_status
0,1000.0,0.1,0.1,0.1,1,BTC/USDT,0,None
